In [21]:
import pandas as pd
import html5lib
import time
from datetime import datetime
import os
import pickle

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [10]:
def initialize_selenium(URL):
    # initialize selenium
    chromedriver = "/Applications/chromedriver" 
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver)
    driver.get(URL)
    
    return driver  

# Generate dictionary to store our data per year
def data_to_dict(years):
    """
    Generate Dictionary that will store our data per year in this format:
    
    Key (Year): Value (Data)
    
    years: int indicating how many years of data will be stored
    """
    data = {}
    CURRENT_YEAR = int(datetime.now().year)
    years_label = range(CURRENT_YEAR-1,CURRENT_YEAR-years,-1)
    
    return years_label, data
    
    
def download_rookie_data(URL, years):
    
    years_label, data = data_to_dict(years)
    driver = initialize_selenium(URL)
    wait = WebDriverWait(driver, 10)
    
    for i in years_label:
        df = pd.read_html(driver.current_url)[0]
        data[i]=df
        prev_year = driver.find_element_by_css_selector("a.button2.prev")
        prev_year.click()
        time.sleep(10)
    
    driver.quit()
    
    return data

In [11]:
data1=download_rookie_data("https://www.basketball-reference.com/leagues/NBA_2018_rookies.html",12)

In [12]:
data1.keys()

dict_keys([2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008])

In [16]:
data1[2008].head(5)

Unnamed: 0_level_0 Unnamed: 1_level_0        Unnamed: 2_level_0  \
                  Rk             Player                     Debut   
0                  1      Arron Afflalo     Nov 1, '07, DET @ MIA   
1                  2       Blake Ahearn  Mar 21, '08, MIA vs. WAS   
2                  3       Lance Allred    Mar 17, '08, CLE @ ORL   
3                  4      Morris Almond   Nov 7, '07, UTA vs. CLE   
4                  5       Joel Anthony  Nov 27, '07, MIA vs. CHA   

  Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0 Unnamed: 6_level_0  \
                 Age                Yrs                  G                 MP   
0                 22                 11                762              20829   
1                 23                  3                 19                226   
2                 26                  1                  3                 10   
3                 22                  3                 38                361   
4                 25                 10                490               7056   

  Totals             ...                  Shooting             Per Game        \
      FG   FGA   3P  ...  TOV    PF   PTS      FG%   3P%   FT%       MP  PS/G   
0   2983  6633  852  ...  852  1481  8198     .450  .386  .825     27.3  10.8   
1     21    77   14  ...   22    27    88     .273  .298  .970     11.9   4.6   
2      1     4    0  ...    0     1     3     .250   NaN  .500      3.3   1.0   
3     43   113    7  ...   19    42   119     .381  .292  .743      9.5   3.1   
4    398   776    0  ...  227   819  1064     .513   NaN  .662     14.4   2.2   

             
   TRB  AST  
0  2.9  1.8  
1  1.2  1.2  
2  0.3  0.0  
3  1.2  0.3  
4  2.8  0.2  

[5 rows x 28 columns]

We need to:
1. Drop the top level of the columns index
2. Remove all but the player name

In [17]:
data={}

for k,v in data1.items():
    temp = v
    temp.columns=temp.columns.droplevel()
    temp = temp[['Player']]
    data[k]=temp


In [19]:
def save_dataset(data,filename):
    with open(filename, 'wb') as w:
        pickle.dump(data,w)

In [22]:
save_dataset(data,"../data/raw/rookies.pickle")